# PL에서 PS DRAM 접근하기

이 챕터에서는 PYNQ Xlnk 클래스가 DDR 메모리에 메모리 버퍼를 할당하는 데 사용된다. 메모리의 실제 주소는 PL로,이 경우에는 IOP로 전달된다. IOP에는 PS DRAM에 대한 연결이 있다. 응용 프로그램은 IOP에서 실행되어 PS DRAM의 메모리 버퍼 내용을 수정한다.

비슷한 방식으로, PL의 다른 IP는 물리적 메모리 포인터를 사용하여 PS DRAM에 액세스 할 수 있다.

### Xlnk의 인스턴스 만들기

In [1]:
from pynq import Xlnk
xlnk = Xlnk()

### mmu의 상태 확인

*cma_stats()*는 Xlnk 인스턴스의 상태를 가져 오는 데 사용할 수 있다. Xlnk은 충분한 공간을 사용할 수있는 경우에만 메모리를 할당 할 수 있다.

In [3]:
xlnk.cma_stats()

{'Buffer Count': 0, 'CMA Memory Available': 76201984, 'CMA Memory Usage': 0}

## 메모리 버퍼 할당
Xlnk의 cma_array() 함수는 인접한 메모리 버퍼를 만든다. 버퍼는 Python에서 사용할 수 있다. 파이썬은 리눅스에서 돌아가고 가상 메모리 주소를 통해 버퍼에 접근 할 것이다. 이것은 Python 레벨에서 사용자에게 보이지 않는다. Xlnk함수 cma_get_phy_addr()을 사용하여 실제 주소를 반환 할 수 있다. 이 물리 주소는 PL의 IP가 PS DRAM의 동일한 메모리 버퍼에 액세스하는 데 사용할 수 있다.

In [4]:
import numpy as np 
py_buffer = xlnk.cma_array(shape=(1000,), dtype=np.int32)

### 메모리 버퍼 주소를 확인하라.

가상 주소는 Linux에서 실행 중인 모든 응용 프로그램에서 사용할 수 있다. 이것은 Python 응용 프로그램이거나 Linux에서 실행중인 C/C++ 또는 다른 응용 프로그램 일 수 있다. 물리적 주소는 오버레이의 IP블록으로 전달 될 수 있다.


### Base Overlay 다운로드

In [5]:
from pynq.overlays.base import BaseOverlay
base = BaseOverlay('base.bit')

## MicroBlaze 프로그램 생성

MicroBlaze에서 실행될 새로운 기능을 위한 C코드는 다음 셀에 제공된다. C 함수 매개 변수는 실제 주소, 길이 및 데이터이다. 이 함수는 각 메모리 위치의 내용을 읽고 각 위치에 오프셋 값 *data*를 추가하여 [*address* : *address* + *length*] 범위의 데이터를 수정한다.

In [6]:
%%microblaze base.ARDUINO
void my_function(unsigned int physical_address, unsigned int length, unsigned int data) {
    int i;
    int *mb_buffer;
    
    // DDR is accessed through a GP port at offset 0x20000000
    mb_buffer = (int *)(physical_address|0x20000000); // Cast to pointer and convert to DDR offset address

    // Write memory buffer in DDR
    for(i=0; i<length; i++){
        mb_buffer[i]= mb_buffer[i] + data;
    }
}

다음 값을 사용하여 버퍼를 초기화하시오.

In [7]:
length = 20 
for i in range(length):
    py_buffer[i] = i

버퍼의 내용을 확인하시오.

In [8]:
py_buffer[0:length]

ContiguousArray([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,
                 14, 15, 16, 17, 18, 19])

## IOP에서 버퍼에 쓰기

*mmu* Xlnk 인스턴스에서 반환 된 실제 포인터 주소와 초기화 값 및 길이를 적어라. IOP 어플리케이션은 메모리 버퍼에 기록한다.

In [9]:
data = 10
my_function(py_buffer.physical_address, length, data)

IOP 어플리케이션이 버퍼를 수정 한 후에 버퍼의 내용을 점검하라. 위의 셀은 다른 값의 데이터와 길이로 재실행 할 수 있다.

In [10]:
py_buffer[0:length]

ContiguousArray([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23,
                 24, 25, 26, 27, 28, 29])